# Введение

После покупки крупного сервиса для чтения книг по подписке перед нами стоит задача анализа базы данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Ознакомление с данными

Для начала импортируем необходимые для работы библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine 


И наладим связь с базой данных:

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем первые строки таблиц для ознакомления со структурой данных. Для этого напишем небольшую функцию:

In [3]:
def heads (df):
    query_1 = 'SELECT * FROM ' + df + ' LIMIT 10'
    return pd.io.sql.read_sql(query_1, con = engine)

    

Первая таблица содержит информацию о книгах:

`book_id` — идентификатор книги;

`author_id` — идентификатор автора;

`title` — название книги;

`num_pages` — количество страниц;

`publication_date` — дата публикации книги;

`publisher_id` — идентификатор издателя.

In [4]:
heads('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


Вторая таблица содержит информацию об авторах:

`author_id` — идентификатор автора;

`author` — имя автора.

In [5]:
heads('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


Третья таблица объединяет данные об издателях:

`publisher_id` — идентификатор издательства;

`publisher` — название издательства;

In [6]:
heads('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


Четвертая таблица содержит данные о пользовательских оценках книг:

`rating_id` — идентификатор оценки;

`book_id` — идентификатор книги;

`username` — имя пользователя, оставившего оценку;

`rating` — оценка книги.

In [7]:
heads('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


И, наконец, последняя, пятая таблица содержит данные о пользовательских обзорах:

`review_id` — идентификатор обзора;

`book_id` — идентификатор книги;

`username` — имя автора обзора;

`text` — текст обзора.

In [8]:
heads('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


### Выводы

Мы ознакомились со структурой базы данных и готовы приступать к составлению запросов.

## Выполнение запросов к базе данных

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [9]:
query = '''
           SELECT COUNT(DISTINCT book_id)
           FROM books 
           WHERE publication_date > '2000-01-01'
        '''

In [10]:
pd.io.sql.read_sql(query, con = engine)

,count
0,819


### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [11]:
query = '''
           SELECT b.book_id, b.title, COUNT(DISTINCT rev.review_id) AS review_count , ROUND(AVG(rt.rating), 2) AS average_rating
           FROM books AS b
           FULL OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
           FULL OUTER  JOIN ratings AS rt ON b.book_id = rt.book_id
           GROUP BY b.book_id
           ORDER BY review_count DESC
           LIMIT 10
        '''

In [12]:
pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_count,average_rating
0,948,Twilight (Twilight #1),7,3.66
1,696,The Da Vinci Code (Robert Langdon #2),6,3.83
2,207,Eat Pray Love,6,3.40
3,627,The Alchemist,6,3.79
4,673,The Catcher in the Rye,6,3.83
5,695,The Curious Incident of the Dog in the Night-Time,6,4.08
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
8,497,Outlander (Outlander #1),6,4.13
9,656,The Book Thief,6,4.26


### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [13]:
query = '''
           SELECT pub.publisher, COUNT(b.book_id) AS book_count
           FROM books AS b
           JOIN publishers AS pub ON b.publisher_id = pub.publisher_id
           WHERE b.num_pages > 50
           GROUP BY pub.publisher
           ORDER BY book_count DESC
           LIMIT 10
        '''

In [14]:
pd.io.sql.read_sql(query, con = engine)

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [15]:
query = '''
           SELECT max_scores.author, ROUND(max_scores.average_rating, 2) AS average_score
           FROM (SELECT author, AVG(rt.rating) AS average_rating
                 FROM authors AS aut
                 JOIN books AS b ON aut.author_id = b.author_id
                 JOIN ratings AS rt ON b.book_id = rt.book_id
                 GROUP BY author
                 HAVING COUNT(rating_id)>50) AS max_scores
           ORDER BY average_rating DESC
           LIMIT 1
        '''

In [16]:
pd.io.sql.read_sql(query, con = engine)

,author,average_score
0,J.K. Rowling/Mary GrandPré,4.29


### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [17]:
query = '''
            SELECT ROUND(AVG(max_texts.count), 2) AS average_review_count
            FROM (SELECT COUNT(review_id)
                  FROM reviews
                  WHERE username IN (SELECT username
                                     FROM ratings AS rt
                                     GROUP BY username 
                                     HAVING COUNT(rating) > 50)
                  GROUP BY username) AS max_texts
        '''

In [18]:
pd.io.sql.read_sql(query, con = engine)

,average_review_count
0,24.33


## Выводы

По итогам составленных запросов мы модем сделать следующие выводы:

* в базе насчитывается 819 книг, изданных после 1 января 2000 года
* больше всего обзоров написано на книгу `Twilight`, но ее средний рейтинг составляет 3,66
* наибольшее количество книг объемом свыше 50 страниц выпущено издательством `Penguin Books` - 42 штуки
* самая высокая средняя оценка книг у автора `J.K. Rowling/Mary GrandPré` - 4,29 (среди книг, получивших более 50 оценок)
* среднее количество обзоров среди самых активных пользователей, поставивших более 50 оценок - 24,33